In [3]:
!git clone https://github.com/andersparslov/BayesianNN-DQR.git
import os
os.chdir('BayesianNN-DQR')

from scipy.stats import norm
from datetime import datetime
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn.metrics import mean_absolute_error as eval_mae
from sklearn.metrics import mean_squared_error as eval_mse
from sklearn import linear_model
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, Lambda
np.seterr(divide='ignore')

def sort_and_order(data):
  ## Sort links by order 
  data, order = sort_links(data, '1973:1412', '7057:7058')
  ## Make a link order column e.g here the neighbouring links for link 1 are 0 and 2.
  data.loc[:,'link_order'] = data['link_ref'].astype('category')
  not_in_list = data['link_order'].cat.categories.difference(order)
  data.loc[:,'link_order'] = data['link_order'].cat.set_categories(np.hstack((order, not_in_list)), ordered=True)
  data.loc[:,'link_order'] = data['link_order'].cat.codes
  ## Add week of day column [Monday, ..., Sunday] = [0, ..., 6]
  data.loc[:, 'Weekday'] = data.index.weekday
  ## Add hour of the time to dataframe
  data.loc[:, 'Hour'] = data.index.hour
  ## Add time of day variables to data frame
  data.loc[:, 'TOD'] = data.Hour.apply(tod_interval)
  data = data.sort_values('link_order')
  return data, order

def skip_row(index, keep_list):
	if (index == 0):
		return False ## Never want to skip the header
	return index not in keep_list

def write_3d(X, filename):
  X_list = X.tolist()
  with open(filename+'.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile, delimiter=',')
      writer.writerows(X_list)

def skip_row(index, keep_list):
	if (index == 0):
		return False ## Never want to skip the header
	return index not in keep_list

def transform(data, means_df, scales_df, order, freq = '15min'):
  tss = { }
  ws = { }
  removed_mean = { }
  removed_scale = { }
  lnk_list = []
  for lnk, data_link in data.groupby('link_ref', sort = False):
      # Link Data Time Indexed
      link_time_ix = pd.DatetimeIndex(data_link.index)
      data_link = data_link.set_index(link_time_ix)
      # Link Reference Data Index
      ix_week = data_link['Weekday'].tolist()
      ix_tod = data_link['TOD'].tolist()
      ## Create multi index for the two lists
      mult_ind = pd.MultiIndex.from_arrays([ix_week, ix_tod])

      link_travel_time_k = data_link['link_travel_time'].resample(freq).mean()
      removed_mean[lnk] = pd.Series(data=means_df[lnk].loc[mult_ind].values, 
                                    index = link_time_ix).resample(freq).mean()
      removed_scale[lnk] = pd.Series(data =scales_df[lnk].loc[mult_ind].values, 
                                      index = link_time_ix).resample(freq).mean()
      tss[lnk] = (link_travel_time_k - removed_mean[lnk].values) / removed_scale[lnk].values
      ws[lnk] = data_link['link_travel_time'].resample(freq).count()
      lnk_list.append(lnk)

  ts = pd.DataFrame(data = tss).fillna(method='pad').fillna(0) 
  df_removed_mean = pd.DataFrame(data = removed_mean, index = ts.index).fillna(method='pad').fillna(method='bfill') 
  df_removed_scale = pd.DataFrame(data = removed_scale, index = ts.index).fillna(method='pad').fillna(method='bfill')    
  w = pd.DataFrame(data = ws).fillna(0) # Link Travel Time Weights, e.g. number of measurements
  return ts[order], df_removed_mean[order], df_removed_scale[order]

def fit_scale(data, order, ref_freq = '15min'):
  means = { }
  scales = { }
  low = { }
  upr = { }

  grouping = data[data['link_travel_time'].notnull()].groupby('link_ref', sort = False)
  for link_ref, data_link in grouping:
      # Fit outlier bounds using MAD
      median = data_link.groupby('Weekday')['link_travel_time'].median()
      error = pd.concat([data_link['Weekday'], np.abs(data_link['link_travel_time'] - median[data_link['Weekday']].values)], axis = 1)
      mad = 1.4826 * error.groupby('Weekday')['link_travel_time'].median()
      _low = median - 3 * mad
      _upr = median + 3 * mad
      mask = (_low[data_link['Weekday']].values < data_link['link_travel_time']) & (data_link['link_travel_time'] < _upr[data_link['Weekday']].values)
      data_link_no = data_link[mask]
      _mean = data_link_no.groupby(['Weekday', 'TOD'])['link_travel_time'].mean()
      means[link_ref] = _mean
      scale = data_link_no.groupby(['Weekday', 'TOD'])['link_travel_time'].std()
      scales[link_ref] = scale

      low[link_ref] = _low
      upr[link_ref] = _upr

  means_df = pd.DataFrame(data=means).interpolate()
  scales_df = pd.DataFrame(data=scales).interpolate()
  low_df = pd.DataFrame(data=low).interpolate()
  upr_df = pd.DataFrame(data=upr).interpolate()

  ## Correct order of links
  means_df = means_df[order]
  scales_df = scales_df[order]
  low_df = low_df[order]
  upr_df = upr_df[order]

  # Fill NaNs    
  means_df = means_df.fillna(method='pad').fillna(method='bfill')
  scales_df = scales_df.fillna(method='pad').fillna(method='bfill')
  low_df = low_df.fillna(method='pad').fillna(method='bfill')
  upr_df = upr_df.fillna(method='pad').fillna(method='bfill')
  
  return means_df, scales_df

def roll(ix, ts, removed_mean, removed_scale, lags, preds):
  X = np.stack([np.roll(ts, i, axis = 0) for i in range(lags, 0, -1)], axis = 1)[lags:-preds,]
  Y = np.stack([np.roll(ts, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
  Y_ix = ix[lags:-preds]
  Y_mean = np.stack([np.roll(removed_mean, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
  Y_scale = np.stack([np.roll(removed_scale, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
  return X, Y, Y_ix, Y_mean, Y_scale

def sort_links(data, start_link, end_link):
  ordered_list = [start_link]
  links = data.loc[:,'link_ref'].unique()
  stop_end = start_link.rpartition(':')[2]
  while True:
      stop_start = stop_end
      for lnk in links:
          if(lnk.rpartition(':')[0] == stop_start):
              if( (lnk in ordered_list) or (lnk == end_link) ):
                  break
              else:
                  ordered_list.append(lnk)
                  stop_end = lnk.rpartition(':')[2]
      if(stop_start == stop_end):
          break
  ordered_list.append(end_link)
  ## Only include links in ordered list.
  data = data[data.loc[:,'link_ref'].isin(ordered_list)]
  return data, ordered_list

def tod_interval(x):
  if(x < 2):
      return 0
  elif(x < 4):
      return 1
  elif(x < 6):
      return 2
  elif(x < 8):
      return 3
  elif(x < 10):
      return 4
  elif(x < 12):
      return 5
  elif(x < 14):
      return 6
  elif(x < 16):
      return 7
  elif(x < 18):
      return 8
  elif(x < 20):
      return 9
  elif(x < 22):
      return 10
  elif(x < 24):
      return 11

def split_df(data, start_train, end_train, end_test):
  data_train = data.loc[start_train:end_train]
  data_test = data.loc[end_train:end_test]
  return data_train, data_test

def split_df_with_val(data, start_train, end_train, end_val, end_test):
  data_train = data.loc[start_train:end_train]
  data_val = data.loc[end_train:end_val]
  data_test = data.loc[end_val:end_test]
  return data_train, data_val, data_test

def drop_remainder(X, y, y_ix, y_mean, y_std, drop):
  return X[:-drop], y[:-drop], y_ix[:-drop], y_mean[:-drop], y_std[:-drop]

def tilted_loss_np_t(q, y, f):
  e = y-f
  # The term inside k.mean is a one line simplification of the first equation
  return np.mean(np.maximum(q*e, (q-1)*e))

Cloning into 'BayesianNN-DQR'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 1765 (delta 84), reused 0 (delta 0), pack-reused 1637
Receiving objects: 100% (1765/1765), 1.29 GiB | 40.54 MiB/s, done.
Resolving deltas: 100% (497/497), done.
Checking out files: 100% (1143/1143), done.


In [0]:
!git clone https://github.com/andersparslov/BayesianNN-DQR.git
import os
os.chdir('BayesianNN-DQR')

Cloning into 'BayesianNN-DQR'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 1738 (delta 68), reused 0 (delta 0), pack-reused 1637
Receiving objects: 100% (1738/1738), 1.29 GiB | 33.02 MiB/s, done.
Resolving deltas: 100% (481/481), done.
Checking out files: 100% (1142/1142), done.


In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, ConvLSTM2D, Flatten, RepeatVector, Reshape, TimeDistributed

## Encoder-decoder LSTM for mean 
def convLstm(num_filters, kernel_length, input_timesteps, num_links, output_timesteps, prob, loss):
    model = Sequential()
    model.add(BatchNormalization(name = 'batch_norm_0', input_shape = (input_timesteps, num_links, 1, 1)))

    model.add(ConvLSTM2D(name ='conv_lstm_1',
                         filters = num_filters, kernel_size = (kernel_length, 1), 
                         padding='same',
                         return_sequences = False))
    model.add(Lambda(lambda x: K.dropout(x, level=prob)))
    model.add(BatchNormalization(name = 'batch_norm_1'))

    model.add(Flatten())
    model.add(RepeatVector(output_timesteps))
    model.add(Reshape((output_timesteps, num_links, 1, num_filters)))

    model.add(ConvLSTM2D(name ='conv_lstm_2',
                         filters = num_filters, kernel_size = (kernel_length, 1), 
                         padding='same',
                         return_sequences = True))
    model.add(Lambda(lambda x: K.dropout(x, level=prob)))
    model.add(TimeDistributed(Dense(units = 1, name = 'dense_1')))
    model.compile(loss = loss, optimizer = 'nadam')
    return model

In [0]:
## Mean parameters (for the selected lag 32 independent mean net)
num_units_mean_l10 = 24
kern_mean_l10 = 11
lags = 10
preds = 1
num_links = 16
prob = 0.20
NUM_MC_SAMPLES = 100
loss = lambda y,f: mse_loss(y, f)
net = convLstm(num_units_mean_l10, kern_mean_l10, lags, num_links, preds, prob, loss)

In [31]:
start = datetime.strptime('19/01/21', "%y/%m/%d")
end   = datetime.strptime('19/04/14', "%y/%m/%d")
period = (end - start).days
period_train_days = 7*4  ## Train on 4 weeks
period_val_days = 7 
period_test_days =  7    ## Test on  1 week
advance_days = 7         ## Advance by 1 week
num_partitions = int((period-period_train_days-period_test_days)/advance_days)
preds = 1
quantiles = np.array([0.025, 0.975, 0.05, 0.95]) 
pred_ints = np.array([0.95,         0.90])
num_links = 16

icp_lr = np.empty((preds, int(len(quantiles)/2), num_partitions))
mil_lr = np.empty((preds, int(len(quantiles)/2), num_partitions))
tradeoff_lr = np.empty((preds, int(len(quantiles)/2), num_partitions))
icp2_lr = np.empty((preds, int(len(quantiles)/2), num_partitions))
mil2_lr = np.empty((preds, int(len(quantiles)/2), num_partitions))
tradeoff2_lr = np.empty((preds, int(len(quantiles)/2), num_partitions))

mse_lr = np.empty((preds, num_partitions))
mape_lr = np.empty((preds, num_partitions))
mse2_lr = np.empty((preds, num_partitions)) ## <- using mean 
mse2 = np.empty((preds, num_partitions)) ## <- using median
mape2_lr = np.empty((preds, num_partitions))

icp_mc_a = np.empty((preds, int(len(quantiles)/2), num_partitions))
mil_mc_a = np.empty((preds, int(len(quantiles)/2), num_partitions))
tradeoff_mc_a = np.empty((preds, int(len(quantiles)/2), num_partitions))
icp_mc_s = np.empty((preds, int(len(quantiles)/2), num_partitions))
mil_mc_s = np.empty((preds, int(len(quantiles)/2), num_partitions))
tradeoff_mc_s = np.empty((preds, int(len(quantiles)/2), num_partitions))
mse_mc = np.empty((preds, num_partitions))
mape_mc = np.empty((preds, num_partitions))

for part in range(num_partitions):
  print("Partition", part+1)
  train_from = part*advance_days
  train_to = period_train_days + part*advance_days
  val_to = period_train_days + part*advance_days + period_val_days
  test_to = period_train_days + part*advance_days + period_test_days

  train_ind = np.arange(train_from, train_to)
  val_ind = np.arange(train_to, val_to)
  test_ind = np.arange(train_to, test_to)
  print("Training on weeks {}".format(np.arange(int((train_ind[6]+1)/7),
                                                int((train_ind[-1]+1)/7)+1)), end=' ')
  keep_train = range(int(2297920*train_ind[ 0]/period), int(2297920*train_ind[-1]/period))
  keep_val = range(int(2297920*train_ind[-1]/period)+1, int(2297920*val_ind[-1]/period))
  keep_test = range(int(2297920*train_ind[-1]/period)+1, int(2297920*test_ind[-1]/period))

  ## Load the part of the dataset we need for training, validation, testing
  data_train = pd.read_csv('data/link_travel_time_local.csv.gz', compression='gzip', 
                            parse_dates = True, index_col = 0,
                            skiprows = lambda x: skip_row(x, keep_train))
  data_val = pd.read_csv('data/link_travel_time_local.csv.gz', compression='gzip', 
                          parse_dates = True, index_col = 0,
                          skiprows = lambda x: skip_row(x, keep_val))
  data_test  = pd.read_csv('data/link_travel_time_local.csv.gz', compression='gzip',
                            parse_dates = True, index_col = 0,
                            skiprows = lambda x: skip_row(x, keep_test))
  ## Sort data by links and add categorical columns TOD, Weekday
  data_train, order = sort_and_order(data_train)
  data_val, order = sort_and_order(data_val)
  data_test, order = sort_and_order(data_test)

  ## Transform datasets using the mean and std for train and val set.
  #means_df_train, scales_df_train = fit_scale(pd.concat([data_train,data_val]), order)
  means_df_train, scales_df_train = fit_scale(data_train, order)
  ts_train_df, mean_train_df, scale_train_df = transform(data_train, 
                                                          means_df_train, 
                                                          scales_df_train, 
                                                          order,
                                                          freq = '15min')
  ts_val_df, mean_val_df, scale_val_df = transform(data_val, 
                                                  means_df_train, 
                                                  scales_df_train, 
                                                  order,
                                                  freq = '15min')
  ts_test_df, mean_test_df, scale_test_df = transform(data_test, 
                                                      means_df_train, 
                                                      scales_df_train, 
                                                      order,
                                                      freq = '15min')
  ## Roll data into timeseries format
  X_train, y_train, y_ix_train, y_mean_train, y_std_train = roll(ts_train_df.index, 
                                                                  ts_train_df.values,
                                                                  mean_train_df.values,
                                                                  scale_train_df.values,
                                                                  lags, 
                                                                  preds)
  X_val, y_val, y_ix_val, y_mean_val, y_std_val = roll(ts_val_df.index, 
                                                    ts_val_df.values,
                                                    mean_val_df.values,
                                                    scale_val_df.values,
                                                    lags, 
                                                    preds)
  X_test, y_test, y_ix_test, y_mean_test, y_std_test = roll(ts_test_df.index, 
                                                            ts_test_df.values, 
                                                            mean_test_df.values,
                                                            scale_test_df.values,
                                                            lags, 
                                                            preds)
  T1 = y_ix_train[0]
  ix_train = np.array((y_ix_train-T1).seconds/60)[:,np.newaxis]
  ix_test  = np.array((y_ix_test-T1).seconds/60)[:,np.newaxis]
  col_names = ["l%d" % (i,) for i in range(lags)]
  col_names.append("y")
  qr_str = "y ~ " + ''.join(["+l%d" % (x,) for x in range(lags)])[1:]

  

  print("Fitting Linear Quantile Regression")
  for t in range(preds):
    print("t + {}".format(t+1))

    y_pred_lr = np.empty((y_test.shape[0], preds, num_links))
    y_pred = np.empty((y_test.shape[0], preds, num_links))
    y_pred_q = np.empty((len(quantiles), y_test.shape[0], preds, num_links))
    icp_lnks = np.empty((preds, num_links, int(len(quantiles)/2)))
    mil_lnks = np.empty((preds, num_links, int(len(quantiles)/2)))
    tradeoff_lnks = np.empty((preds, num_links, int(len(quantiles)/2)))
    mse_lnks = np.empty((preds, num_links, int(len(quantiles)/2)))
    mape_lnks = np.empty((preds, num_links, int(len(quantiles)/2)))

    for lnk in range(num_links):
      regr = linear_model.LinearRegression()
      regr.fit(X_train[:,:,lnk], y_train[:,t,lnk])
      y_pred_lr[:,t,lnk] = regr.predict(X_test[:,:,lnk])*y_std_test[:,t,lnk] + y_mean_test[:,t,lnk]
      
      for q, quan in enumerate(quantiles):
        data = pd.DataFrame(data = np.hstack([X_train[:,:,lnk], y_train[:,t,lnk:(lnk+1)]]), columns = col_names)
        data_test = pd.DataFrame(data = np.hstack([X_test[:,:,lnk]]), columns = col_names[:lags])
        qr = smf.quantreg(qr_str, data)
        res = qr.fit(q=quan)
        y_pred_q[q,:,t,lnk] = res.predict(data_test)
      res = qr.fit(q=0.5)
      y_pred[:,t,lnk] = res.predict(data_test)

      ## Quantile evaluation
      for i in range(int(len(quantiles)/2)):
        q1 = y_pred_q[2*i,:,t,lnk]
        q2 = y_pred_q[2*i+1,:,t,lnk]
        q1_back = q1*y_std_test[:,0,lnk]+y_mean_test[:,0,lnk]
        q2_back = q2*y_std_test[:,0,lnk]+y_mean_test[:,0,lnk]
        icp_lnks[t, lnk, i] = 1-(np.sum(y_test[:,t,lnk] < q1)+np.sum(y_test[:,t,lnk] > q2))/len(y_test)
        mil_lnks[t, lnk, i] = np.sum(np.maximum(0, q2_back - q1_back)) / len(y_test)
        tradeoff_lnks[t, lnk, i] = (icp_lnks[t,lnk,i] - pred_ints[i])*mil_lnks[t, lnk, i]
      y_test_lnk = y_test[:,t,lnk]*y_std_test[:,t,lnk] + y_mean_test[:,t,lnk]
      y_pred_lnk = y_pred[:,t,lnk]*y_std_test[:,t,lnk] + y_mean_test[:,t,lnk]
      mse_lnk = eval_mse(y_test_lnk/60, y_pred_lnk/60)
      mape_lnk = np.mean(np.abs((y_pred_lnk/60 - y_test_lnk/60)/(y_test_lnk/60)))
    
    mse_lr[t, part] = np.mean(mse_lnk)
    mape_lr[t, part] = np.mean(mape_lnk)
    for i in range(int(len(quantiles)/2)):
      icp_lr[t, i, part] = np.mean(icp_lnks[t, :, i])
      mil_lr[t, i, part] = np.mean(mil_lnks[t, :, i])
      tradeoff_lr[t, i, part] = np.mean(tradeoff_lnks[t, :, i])

      q1 = y_pred_q[2*i,:,t,:]*y_std_test[:,t,:] + y_mean_test[:,t,:]
      q2 = y_pred_q[2*i+1,:,t,:]*y_std_test[:,t,:] + y_mean_test[:,t,:]
      q1_all = np.sum(q1, axis=1)
      q2_all = np.sum(q2, axis=1)
      y_test_all = np.sum(y_test[:,t]*y_std_test[:,t,:] + y_mean_test[:,t,:], axis=1)
      icp2_lr[t,  i, part] = 1-(np.sum(y_test_all < q1_all)+np.sum(y_test_all> q2_all))/len(y_test)
      mil2_lr[t,  i, part] = np.sum(np.maximum(0, q2 - q1)) / len(y_test)
      tradeoff2_lr[t, i, part] = (icp2_lr[t,i,part] - pred_ints[i])*mil2_lr[t,i,part]

    y_test_all = np.sum(y_test[:,t]*y_std_test[:,t] + y_mean_test[:,t], axis=1)
    y_pred_all = np.sum(y_pred[:,t]*y_std_test[:,t] + y_mean_test[:,t], axis=1)
    y_pred_all_lr = np.sum(y_pred_lr[:,t], axis=1)
    mse2_lr[t, part] = eval_mse(y_test_all/60, y_pred_all_lr/60)
    mse2[t, part] = eval_mse(y_test_all/60, y_pred_all/60)
    mape2_lr[t, part] = np.mean(np.abs((y_pred_all/60 - y_test_all/60)/(y_test_all/60)))
    
    print(" LR ICP (95%) {} MSE mean (route) MSE med (route) {} MAPE {} (route)".format(icp_lr[t, 0, part], mse2_lr[t, part], mse2[t, part], mape2_lr[t, part]))
    
    X_train = X_train[:,:,:,np.newaxis,np.newaxis]
    X_val = X_val[:,:,:,np.newaxis,np.newaxis]
    X_test = X_test[:,:,:,np.newaxis,np.newaxis]
    y_train = y_train[:,:,:,np.newaxis,np.newaxis]
    y_val = y_val[:,:,:,np.newaxis,np.newaxis]
    y_test = y_test[:,:,:,np.newaxis,np.newaxis]
    
    print("Fitting model for MC Dropout")
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',  patience=5, mode='min')
    check = tf.keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/ModelCheckpoint/baseline_check.hdf5",  
                                               monitor='val_loss', mode='min', 
                                               save_best_only=True)
    net.fit(X_train, y_train, batch_size=80, epochs=50, validation_data=(X_val, y_val), callbacks = [es, check])
    net.load_weights("/content/gdrive/My Drive/ModelCheckpoint/baseline_check.hdf5")

    print("Drawing samples", end='')
    # make predictions
    predictions = np.zeros((NUM_MC_SAMPLES, len(X_test), num_links))
    for s in range(NUM_MC_SAMPLES):
      predictions[s,:,:] = np.squeeze(net.predict(X_test))
      if (s+1) % (int(NUM_MC_SAMPLES/3)) == 0:
        print(".", end = '')
    print("")
    # make predictions
    trues_sigma = y_val[:,0,:,0,0]
    predictions_sigma = np.zeros((NUM_MC_SAMPLES, len(X_val), num_links))
    for s in range(NUM_MC_SAMPLES):
      predictions_sigma[s,:,:] = np.squeeze(net.predict(X_val))

    # select optimal value of alpha based on log probability on validation set
    alphas = np.zeros((num_links,))
    for lnk in range(num_links):
      means = np.mean(predictions_sigma[:,:,lnk], axis=0)
      vars_ = np.var(predictions_sigma[:,:,lnk], axis=0)
      
      best_log_prob = -np.inf
      for alpha in np.arange(0.0,1.0,0.01):
        log_prob = 0.0
        for n in range(len(y_val)):
          log_prob += np.log(norm.pdf(trues_sigma[n,lnk], loc=means[n], scale=np.sqrt(vars_[n]+alpha)))
        if log_prob > best_log_prob:
          best_log_prob = log_prob
          alphas[lnk] = alpha

    y_pred =     np.empty( (y_test.shape[0], num_links) )
    y_pred_q =   np.empty( (len(quantiles), y_test.shape[0], num_links) )
    icp_lnks_a = np.empty( (preds,num_links,int(len(quantiles)/2))) 
    mil_lnks_a = np.empty( (preds,num_links,int(len(quantiles)/2)))
    tradeoff_lnks_a = np.empty( (preds,num_links,int(len(quantiles)/2)))
    icp_lnks_s = np.empty( (preds,num_links,int(len(quantiles)/2)))
    mil_lnks_s = np.empty( (preds,num_links,int(len(quantiles)/2)))
    tradeoff_lnks_s = np.empty( (preds,num_links,int(len(quantiles)/2)))
    for lnk in range(num_links):
      for i in range(int(len(quantiles)/2)):
        trues = y_test[:,0,lnk,0,0]
        preds_mc = np.mean(predictions, axis=0)[:,lnk]
        
        sigma2_est = np.mean((np.mean(predictions_sigma, axis=0) - trues_sigma)**2, axis=0)
        var_dl = np.var(predictions, axis=0)[:,lnk]
        
        inv_cdf_upr = norm.ppf(quantiles[2*i+1])
        inv_cdf_lwr = norm.ppf(quantiles[2*i])
        # here we use different alternatives for calibrating the uncertainty estimates produced by MCdropout
        q1 = np.mean(predictions, axis=0)[:,lnk] + inv_cdf_lwr*np.sqrt(var_dl + alphas[lnk]) 
        q2 = np.mean(predictions, axis=0)[:,lnk] + inv_cdf_upr*np.sqrt(var_dl + alphas[lnk])
        q1_back = q1*y_std_test[:,0,lnk]+y_mean_test[:,0,lnk]
        q2_back = q2*y_std_test[:,0,lnk]+y_mean_test[:,0,lnk]
        icp_lnks_a[t,lnk,i] = 1-(np.sum(y_test[:,0,lnk,0,0] < q1) + np.sum(y_test[:,0,lnk,0,0] > q2))/len(y_test)
        mil_lnks_a[t,lnk,i] = np.sum(np.maximum(0, q2_back - q1_back))/len(y_test)
        tradeoff_lnks_a[t,lnk,i] = (icp_lnks[t,lnk,i]-pred_ints[i])*mil_lnks[t,lnk,i]

        q1 = np.mean(predictions, axis=0)[:,i] + inv_cdf_lwr*np.sqrt(var_dl + sigma2_est[i])
        q2 = np.mean(predictions, axis=0)[:,i] + inv_cdf_upr*np.sqrt(var_dl + sigma2_est[i])
        q1_back = q1*y_std_test[:,0,lnk]+y_mean_test[:,0,lnk]
        q2_back = q2*y_std_test[:,0,lnk]+y_mean_test[:,0,lnk]
        icp_lnks_s[t,lnk,i] = 1-(np.sum(y_test[:,0,lnk,0,0] < q1) + np.sum(y_test[:,0,lnk,0,0] > q2))/len(y_test)
        mil_lnks_s[t,lnk,i] = np.sum(np.maximum(0, q2_back - q1_back))/len(y_test)
        tradeoff_lnks_s[t,lnk,i] = (icp_lnks[t,lnk,i]-pred_ints[i])*mil_lnks[t,lnk,i]

    for i in range(int(len(quantiles)/2)):
      icp_mc_s[t, i, part] = np.mean(icp_lnks_s[t,:,i])
      mil_mc_s[t, i, part] = np.mean(mil_lnks_s[t,:,i])
      tradeoff_mc_s = np.mean(tradeoff_lnks_s[t,:,i])
      icp_mc_a[t, i, part] = np.mean(icp_lnks_a[t,:,i])
      mil_mc_a[t, i, part] = np.mean(mil_lnks_a[t,:,i])
      tradeoff_mc_a = np.mean(tradeoff_lnks_a[t,:,i])
    y_pred_all = np.sum(np.squeeze(np.mean(predictions, axis=0))*y_std_test[:,t] + y_mean_test[:,t], axis=1)
    y_test_all = np.sum(y_test[:,t,:,0,0]*y_std_test[:,t] + y_mean_test[:,t], axis=1)
    mse_mc[t,part] = eval_mse(y_test_all/60, y_pred_all/60)
    mape_mc[t,part] = np.sum(np.abs((y_pred_all/60 - y_test_all/60)/(y_test_all/60)))/len(y_test)
    print(" MC ICP [alpha] (95%) {} MSE (route) {} MAPE {} (route)".format(icp_mc_a[t, 0, part], mse_mc[t, part], mape_mc[t, part]))
    print(" MC ICP [sigma] (95%) {} MSE (route) {} MAPE {} (route)".format(icp_mc_s[t, 0, part], mse_mc[t, part], mape_mc[t, part]))

Partition 1
Training on weeks [1 2 3 4] Fitting Linear Quantile Regression
t + 1


/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)


 LR ICP (95%) 0.932542872454448 MSE mean (route) MSE med (route) 1.3163599680929685 MAPE 1.5115572216742754 (route)
Fitting model for MC Dropout
Train on 3652 samples, validate on 933 samples
Epoch 1/50
3652/3652 [==============================] - 13s 4ms/sample - loss: 2.4542 - val_loss: 1.0443
Epoch 2/50
3652/3652 [==============================] - 9s 3ms/sample - loss: 2.2764 - val_loss: 1.0384
Epoch 3/50
3652/3652 [==============================] - 10s 3ms/sample - loss: 2.2287 - val_loss: 1.0277
Epoch 4/50
3652/3652 [==============================] - 10s 3ms/sample - loss: 2.1974 - val_loss: 1.0223
Epoch 5/50
3652/3652 [==============================] - 10s 3ms/sample - loss: 2.1529 - val_loss: 1.0100
Epoch 6/50
3652/3652 [==============================] - 11s 3ms/sample - loss: 2.1285 - val_loss: 0.9970
Epoch 7/50
3652/3652 [==============================] - 11s 3ms/sample - loss: 2.0892 - val_loss: 0.9903
Epoch 8/50
3652/3652 [==============================] - 11s 3ms/sample - l

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)


 LR ICP (95%) 0.9317208904109588 MSE mean (route) MSE med (route) 1.537182268263054 MAPE 1.8892116493708175 (route)
Fitting model for MC Dropout
Train on 3607 samples, validate on 876 samples
Epoch 1/50
3607/3607 [==============================] - 9s 3ms/sample - loss: 2.0791 - val_loss: 1.1297
Epoch 2/50
3607/3607 [==============================] - 9s 2ms/sample - loss: 2.0685 - val_loss: 1.1311
Epoch 3/50
3607/3607 [==============================] - 9s 3ms/sample - loss: 2.0272 - val_loss: 1.1287
Epoch 4/50
3607/3607 [==============================] - 9s 2ms/sample - loss: 2.0288 - val_loss: 1.1264
Epoch 5/50
3607/3607 [==============================] - 9s 2ms/sample - loss: 2.0121 - val_loss: 1.1277
Epoch 6/50
3607/3607 [==============================] - 9s 2ms/sample - loss: 1.9853 - val_loss: 1.1290
Epoch 7/50
3607/3607 [==============================] - 9s 2ms/sample - loss: 1.9781 - val_loss: 1.1408
Epoch 8/50
3607/3607 [==============================] - 9s 2ms/sample - loss: 1.

...

 LR ICP (95%) 0.9311827956989247 MSE mean (route) MSE med (route) 2.6810476263629934 MAPE 3.3799560689022266 (route)
Fitting model for MC Dropout
Train on 3606 samples, validate on 930 samples
Epoch 1/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.1080 - val_loss: 1.4169
Epoch 2/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.1107 - val_loss: 1.4094
Epoch 3/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.1058 - val_loss: 1.4000
Epoch 4/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.1047 - val_loss: 1.4030
Epoch 5/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.0977 - val_loss: 1.3920
Epoch 6/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.0892 - val_loss: 1.4108
Epoch 7/50
3606/3606 [==============================] - 10s 3ms/sample - loss: 1.1003 - val_loss: 1.4080
Epoch 8/50
3606/3606 [==============================] - 10s 3ms/sample -

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)


 LR ICP (95%) 0.9336890243902438 MSE mean (route) MSE med (route) 1.9604162083942078 MAPE 2.310349081893488 (route)
Fitting model for MC Dropout
Train on 3626 samples, validate on 902 samples
Epoch 1/50
3626/3626 [==============================] - 9s 3ms/sample - loss: 1.1376 - val_loss: 1.3067
Epoch 2/50
3626/3626 [==============================] - 9s 2ms/sample - loss: 1.1253 - val_loss: 1.3164
Epoch 3/50
3626/3626 [==============================] - 9s 3ms/sample - loss: 1.1149 - val_loss: 1.3053
Epoch 4/50
3626/3626 [==============================] - 9s 2ms/sample - loss: 1.1151 - val_loss: 1.3155
Epoch 5/50
3626/3626 [==============================] - 9s 2ms/sample - loss: 1.1108 - val_loss: 1.3122
Epoch 6/50
3626/3626 [==============================] - 9s 2ms/sample - loss: 1.1088 - val_loss: 1.3068
Epoch 7/50
3626/3626 [==============================] - 9s 3ms/sample - loss: 1.1078 - val_loss: 1.2875
Epoch 8/50
3626/3626 [==============================] - 9s 2ms/sample - loss: 1.

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/quantile_regression.py:192: IterationLimitWarning: Maximum number of iterations (1000) reached.
  ") reached.", IterationLimitWarning)
/usr/local/lib/

 LR ICP (95%) 0.9170609884332281 MSE mean (route) MSE med (route) 6.848356664818052 MAPE 9.519370332446059 (route)
Fitting model for MC Dropout
Train on 3601 samples, validate on 951 samples
Epoch 1/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1789 - val_loss: 6.9876
Epoch 2/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1658 - val_loss: 7.0048
Epoch 3/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1710 - val_loss: 6.9184
Epoch 4/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1609 - val_loss: 6.9098
Epoch 5/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1545 - val_loss: 6.9566
Epoch 6/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1632 - val_loss: 6.8887
Epoch 7/50
3601/3601 [==============================] - 10s 3ms/sample - loss: 1.1608 - val_loss: 6.9463
Epoch 8/50
3601/3601 [==============================] - 10s 3ms/sample - l

In [48]:
for i in range(int(len(quantiles)/2)):
  print(pred_ints[i])
  print(" LR")
  print("   ICP %.3f" % np.mean(icp_lr[:,i],axis=1))
  print("   MIL %.3f" % (np.mean(mil_lr[:,i],axis=1)/60))
  print("   ICP (route) %.3f" % np.mean(icp2_lr[:,i],axis=1))
  print("   MIL (route) %.3f" % np.mean(mil2_lr[:,i],axis=1))

  print(" MC DROPOUT (Gal)")
  print("   ICP %.3f" % np.mean(icp_mc_a[:,i],axis=1))
  print("   MIL %.3f" % (np.mean(mil_mc_a[:,i],axis=1)/60))
  print(" MC DROPOUT (Zhu)")
  print("   ICP %.5f" % np.mean(icp_mc_s[:,i],axis=1))
  print("   MIL %.5f" % (np.mean(mil_mc_s[:,i],axis=1)/60))

print("LR")
print("   MSE %.3f" % np.mean(mse_lr,axis=1))
print("   MAPE %.3f" % (100*np.mean(mape_lr,axis=1)))
print("   MSE (route) %.3f" % np.mean(mse2,axis=1))
print("   MAPE (route) %.3f" % (100*np.mean(mape2_lr,axis=1)))
print("MC")
print("   MSE %.3f" % np.mean(mse_mc,axis=1))
print("   MAPE %.3f" % (100*np.mean(mape_mc,axis=1)))

0.95
 LR
   ICP 0.929
   MIL 0.914
   ICP (route) 0.999
   MIL (route) 877.380
 MC DROPOUT (Gal)
   ICP 0.934
   MIL 0.938
 MC DROPOUT (Zhu)
   ICP 0.94187
   MIL 0.95929
0.9
 LR
   ICP 0.877
   MIL 0.727
   ICP (route) 0.997
   MIL (route) 698.281
 MC DROPOUT (Gal)
   ICP 0.902
   MIL 0.787
 MC DROPOUT (Zhu)
   ICP 0.94194
   MIL 0.97482
LR
   MSE 0.065
   MAPE 11.548
   MSE (route) 3.551
   MAPE (route) 5.276
MC
   MSE 3.229
   MAPE 5.103
